In [54]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt



In [55]:
df = pd.read_csv("cleaning.csv")
df

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,12/04/1999,1,"£729,000",apartment
1,Ashford,05/08/2017,unknown,"£699,000",semi-detached
2,Stratford-on-Avon,29/03/2012,3,NaN,detached
3,Canterbury,01/07/2009,2,"£529,000",teraced
4,Camden,16/12/2001,1,"£616,000",apartment
5,Rugby,01/03/2003,-,"£247,000",detached
6,Hampstead,05/03/2016,2,£0,terraced
7,Clapham,05/07/2001,363,"£543,000",apartment
8,Stratford-on-Avon,10th May 2010,3,"£420,000",detached
9,Camden,16/12/2001,1,"£616,000",apartment


Examine our data
Let's first understand a few things about our data.

What data types has Pandas assigned to the features?

In [56]:
df.dtypes

location              object
date_of_sale          object
number_of_bedrooms    object
price                 object
type                  object
dtype: object

In [57]:
# Check nulls?
df.isnull().mean()

location              0.0
date_of_sale          0.0
number_of_bedrooms    0.0
price                 0.1
type                  0.0
dtype: float64

Process location
The location column looks fine. Its data type is object (i.e. string). And there are no nulls. There is nothing we need to do.

Process date_of_sale
The date_of_sale is clearly a date, but has come through as an object (string). We need to convert it to a date otherwise Pandas date functionality won't work.

In [58]:
## Convert date_of_sale from string to datetime\

df["date_of_sale"]= pd.to_datetime(df["date_of_sale"])
df["date_of_sale"]

0   1999-12-04
1   2017-05-08
2   2012-03-29
3   2009-01-07
4   2001-12-16
5   2003-01-03
6   2016-05-03
7   2001-05-07
8   2010-05-10
9   2001-12-16
Name: date_of_sale, dtype: datetime64[ns]

Process number_of_bedrooms
This should be numeric, but has come through as object (string) because there are some non-numeric values. Let's get a list of such values:

In [59]:
# Find non-numerics in number_of_bedrooms

non_nums = df[~df["number_of_bedrooms"].str.isnumeric()]["number_of_bedrooms"].unique()
non_nums

array(['unknown', '-'], dtype=object)

In [60]:
df["number_of_bedrooms"]= df["number_of_bedrooms"].replace(non_nums,np.nan)
df["number_of_bedrooms"]

0      1
1    NaN
2      3
3      2
4      1
5    NaN
6      2
7    363
8      3
9      1
Name: number_of_bedrooms, dtype: object

In [61]:
#convert to numeric

df["number_of_bedrooms"]= pd.to_numeric(df["number_of_bedrooms"])
df["number_of_bedrooms"]

0      1.0
1      NaN
2      3.0
3      2.0
4      1.0
5      NaN
6      2.0
7    363.0
8      3.0
9      1.0
Name: number_of_bedrooms, dtype: float64

Process price
Again for price, we would hope this would be numeric, but it contains characters such as £ and the comma that are not numeric, which must be removed. Let's do that.

In [62]:
# Remove "$"

df["price"]= df["price"].apply(lambda x: x.replace("£","") if type(x) is str else x)
df["price"]= df["price"].apply(lambda x: x.replace(",","") if type(x) is str else x)
df["price"]

0    729000
1    699000
2       NaN
3    529000
4    616000
5    247000
6         0
7    543000
8    420000
9    616000
Name: price, dtype: object

In [63]:
#convert to numeric 
df["price"]=pd.to_numeric(df["price"])
df["price"]

0    729000.0
1    699000.0
2         NaN
3    529000.0
4    616000.0
5    247000.0
6         0.0
7    543000.0
8    420000.0
9    616000.0
Name: price, dtype: float64

In [64]:
df["price"]= df["price"].replace(0,np.nan)
df["price"]

0    729000.0
1    699000.0
2         NaN
3    529000.0
4    616000.0
5    247000.0
6         NaN
7    543000.0
8    420000.0
9    616000.0
Name: price, dtype: float64

In [65]:
df["type"].unique()

array(['apartment', 'semi-detached', 'detached', 'teraced', 'terraced'],
      dtype=object)

In [66]:
df["type"]=df["type"].replace(["tercaed",["terrcard"]])
df["type"]

0        apartment
1    semi-detached
2         detached
3          teraced
4        apartment
5         detached
6         terraced
7        apartment
8         detached
9        apartment
Name: type, dtype: object

In [67]:
df 

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,1999-12-04,1.0,729000.0,apartment
1,Ashford,2017-05-08,NaN,699000.0,semi-detached
2,Stratford-on-Avon,2012-03-29,3.0,NaN,detached
3,Canterbury,2009-01-07,2.0,529000.0,teraced
4,Camden,2001-12-16,1.0,616000.0,apartment
5,Rugby,2003-01-03,NaN,247000.0,detached
6,Hampstead,2016-05-03,2.0,NaN,terraced
7,Clapham,2001-05-07,363.0,543000.0,apartment
8,Stratford-on-Avon,2010-05-10,3.0,420000.0,detached
9,Camden,2001-12-16,1.0,616000.0,apartment


Dung Ham find_outliers de kiem gia tri bi trung lap

In [68]:
def find_outliers(feature):
    '''Return a list of outliers in the data'''

    # Temporarily replace nulls with mean so they don't cause an error
    feature = feature.fillna(feature.mean()) 

    # Compute the quartiles
    quartile_1, quartile_3 = np.percentile(feature, [25, 75])

    # Compute the inter-quartile range
    iqr = quartile_3 - quartile_1

    # Compute the outlier boundaries
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)

    # Return rows where the feature is outside the outlier boundaries
    return np.where((feature > upper_bound) | (feature < lower_bound))

In [69]:
find_outliers(df.number_of_bedrooms.fillna(df.number_of_bedrooms.mean()))

(array([7], dtype=int64),)

In [70]:

find_outliers(df.price.fillna(df.price.mean()))

(array([5], dtype=int64),)

In [71]:
df.loc[7,'number_of_bedrooms'] = np.NaN
df

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,1999-12-04,1.0,729000.0,apartment
1,Ashford,2017-05-08,NaN,699000.0,semi-detached
2,Stratford-on-Avon,2012-03-29,3.0,NaN,detached
3,Canterbury,2009-01-07,2.0,529000.0,teraced
4,Camden,2001-12-16,1.0,616000.0,apartment
5,Rugby,2003-01-03,NaN,247000.0,detached
6,Hampstead,2016-05-03,2.0,NaN,terraced
7,Clapham,2001-05-07,NaN,543000.0,apartment
8,Stratford-on-Avon,2010-05-10,3.0,420000.0,detached
9,Camden,2001-12-16,1.0,616000.0,apartment


In [72]:
# Find duplicates
df[df.duplicated()]

,location,date_of_sale,number_of_bedrooms,price,type
9,Camden,2001-12-16,1.0,616000.0,apartment


In [85]:
df = df.drop_duplicates()

In [86]:
df

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,1999-12-04,1.0,729000.000000,apartment
1,Ashford,2017-05-08,2.0,699000.000000,semi-detached
2,Stratford-on-Avon,2012-03-29,3.0,540428.571429,detached
3,Canterbury,2009-01-07,2.0,529000.000000,teraced
4,Camden,2001-12-16,1.0,616000.000000,apartment
5,Rugby,2003-01-03,2.0,247000.000000,detached
6,Hampstead,2016-05-03,2.0,540428.571429,terraced
7,Clapham,2001-05-07,2.0,543000.000000,apartment
8,Stratford-on-Avon,2010-05-10,3.0,420000.000000,detached


In [75]:
df.isnull().mean()

location              0.000000
date_of_sale          0.000000
number_of_bedrooms    0.333333
price                 0.222222
type                  0.000000
dtype: float64

In [76]:
# Remove columns containing nulls
df_dropcols = df.dropna(axis=1)
df_dropcols

,location,date_of_sale,type
0,Clapham,1999-12-04,apartment
1,Ashford,2017-05-08,semi-detached
2,Stratford-on-Avon,2012-03-29,detached
3,Canterbury,2009-01-07,teraced
4,Camden,2001-12-16,apartment
5,Rugby,2003-01-03,detached
6,Hampstead,2016-05-03,terraced
7,Clapham,2001-05-07,apartment
8,Stratford-on-Avon,2010-05-10,detached


In [77]:
# Remove rows containing nulls
df_droprows = df.dropna()
df_droprows

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,1999-12-04,1.0,729000.0,apartment
3,Canterbury,2009-01-07,2.0,529000.0,teraced
4,Camden,2001-12-16,1.0,616000.0,apartment
8,Stratford-on-Avon,2010-05-10,3.0,420000.0,detached


In [80]:
mean = df["price"].mean()       # calculate the mean for the column
df["price"]=df["price"].fillna(value=mean)  # replace nulls with the mean
df


C:\Users\minhd\AppData\Local\Temp\ipykernel_7896\2482384592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["price"]=df["price"].fillna(value=mean)  # replace nulls with the mean


,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,1999-12-04,1.0,729000.000000,apartment
1,Ashford,2017-05-08,NaN,699000.000000,semi-detached
2,Stratford-on-Avon,2012-03-29,3.0,540428.571429,detached
3,Canterbury,2009-01-07,2.0,529000.000000,teraced
4,Camden,2001-12-16,1.0,616000.000000,apartment
5,Rugby,2003-01-03,NaN,247000.000000,detached
6,Hampstead,2016-05-03,2.0,540428.571429,terraced
7,Clapham,2001-05-07,NaN,543000.000000,apartment
8,Stratford-on-Avon,2010-05-10,3.0,420000.000000,detached


In [81]:
median = df["price"].median()   # calculate the median for the column
df["price"].fillna(value=median)  # replace nulls with the mean

0    729000.000000
1    699000.000000
2    540428.571429
3    529000.000000
4    616000.000000
5    247000.000000
6    540428.571429
7    543000.000000
8    420000.000000
Name: price, dtype: float64

In [83]:
mean = round(df["number_of_bedrooms"].mean())     # calculate the mean for the column
df["number_of_bedrooms"] = df["number_of_bedrooms"].fillna(value=mean)  # replace nulls with the mean
df

C:\Users\minhd\AppData\Local\Temp\ipykernel_7896\1413745270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["number_of_bedrooms"] = df["number_of_bedrooms"].fillna(value=mean)  # replace nulls with the mean


,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,1999-12-04,1.0,729000.000000,apartment
1,Ashford,2017-05-08,2.0,699000.000000,semi-detached
2,Stratford-on-Avon,2012-03-29,3.0,540428.571429,detached
3,Canterbury,2009-01-07,2.0,529000.000000,teraced
4,Camden,2001-12-16,1.0,616000.000000,apartment
5,Rugby,2003-01-03,2.0,247000.000000,detached
6,Hampstead,2016-05-03,2.0,540428.571429,terraced
7,Clapham,2001-05-07,2.0,543000.000000,apartment
8,Stratford-on-Avon,2010-05-10,3.0,420000.000000,detached


In [84]:
df

,location,date_of_sale,number_of_bedrooms,price,type
0,Clapham,1999-12-04,1.0,729000.000000,apartment
1,Ashford,2017-05-08,2.0,699000.000000,semi-detached
2,Stratford-on-Avon,2012-03-29,3.0,540428.571429,detached
3,Canterbury,2009-01-07,2.0,529000.000000,teraced
4,Camden,2001-12-16,1.0,616000.000000,apartment
5,Rugby,2003-01-03,2.0,247000.000000,detached
6,Hampstead,2016-05-03,2.0,540428.571429,terraced
7,Clapham,2001-05-07,2.0,543000.000000,apartment
8,Stratford-on-Avon,2010-05-10,3.0,420000.000000,detached


In [37]:
# Compute year
df['year'] = df.date_of_sale.dt.year
df

C:\Users\minhd\AppData\Local\Temp\ipykernel_7896\734048357.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df.date_of_sale.dt.year


,location,date_of_sale,number_of_bedrooms,price,type,year
0,Clapham,1999-12-04,1.0,729000.0,apartment,1999
1,Ashford,2017-05-08,2.0,699000.0,semi-detached,2017
2,Stratford-on-Avon,2012-03-29,3.0,NaN,detached,2012
3,Canterbury,2009-01-07,2.0,529000.0,teraced,2009
4,Camden,2001-12-16,1.0,616000.0,apartment,2001
5,Rugby,2003-01-03,2.0,247000.0,detached,2003
6,Hampstead,2016-05-03,2.0,NaN,terraced,2016
7,Clapham,2001-05-07,2.0,543000.0,apartment,2001
8,Stratford-on-Avon,2010-05-10,3.0,420000.0,detached,2010


In [38]:
df.groupby(['number_of_bedrooms'])['price'].mean()

number_of_bedrooms
1.0    672500.0
2.0    504500.0
3.0    420000.0
Name: price, dtype: float64

In [39]:
df['price'] = df.groupby(['number_of_bedrooms'])['price'].transform(lambda x: x.fillna(x.mean()))

C:\Users\minhd\AppData\Local\Temp\ipykernel_7896\2366941064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df.groupby(['number_of_bedrooms'])['price'].transform(lambda x: x.fillna(x.mean()))


In [40]:
df

,location,date_of_sale,number_of_bedrooms,price,type,year
0,Clapham,1999-12-04,1.0,729000.0,apartment,1999
1,Ashford,2017-05-08,2.0,699000.0,semi-detached,2017
2,Stratford-on-Avon,2012-03-29,3.0,420000.0,detached,2012
3,Canterbury,2009-01-07,2.0,529000.0,teraced,2009
4,Camden,2001-12-16,1.0,616000.0,apartment,2001
5,Rugby,2003-01-03,2.0,247000.0,detached,2003
6,Hampstead,2016-05-03,2.0,504500.0,terraced,2016
7,Clapham,2001-05-07,2.0,543000.0,apartment,2001
8,Stratford-on-Avon,2010-05-10,3.0,420000.0,detached,2010
